In [1]:
from custom_clss_and_fncs import *
from cobra.io import load_json_model

model                                 = load_json_model("./COBRA_models/GEM_Recon2_thermocurated_redHUMAN.json")
feature_data = pd.read_parquet("./results/data/oversampled_augmented_metabolite_data.parquet.gzip")#
feature_names                         = pd.read_csv("./results/data/metabolite_names.csv")

/DeepenData/.miniconda/envs/geo/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


### add concentratios as features

In [2]:
grafo_nx                              = cobra_a_networkx(model)

node_list = list(grafo_nx.nodes)
feature_data.rename(
    columns=feature_names.set_index("Simbolo_traductor")["Recon3_ID"].to_dict(), 
    inplace=True
)

number_to_fill_in = int(1)

feature_data[
    [item for item in node_list if item not in feature_data.columns.tolist()]
] = number_to_fill_in

# Re-empaca todo este vector como un diccionario de atributos
feature_dict = feature_data.to_dict(orient="list")  # desde Pandas
feature_dict = {key: {"x": feature_dict[key]} for key in feature_dict}

nx.set_node_attributes(grafo_nx, feature_dict)

assert feature_data['phe_L_c'].tolist() == grafo_nx.nodes['phe_L_c']['x']
assert np.unique(grafo_nx.nodes(data=True)['r0399']['x']) == number_to_fill_in
assert len(grafo_nx.nodes(data=True)['r0399']['x']) == len(grafo_nx.nodes(data=True)['phe_L_c']['x'])

In [3]:
pyg_graph       = from_networkx(copy.deepcopy(nx.Graph(grafo_nx)))


In [4]:
x_attribute     = nx.get_node_attributes(grafo_nx, "x")
longest_feature = max(len(v) for k,v in x_attribute.items())
assert pyg_graph.x.shape[1]  == pyg_graph.num_features == longest_feature # == flux_samples.shape[0]
target_node = 'phe_L_c'
producto_idx = list(grafo_nx.nodes()).index(target_node)
producto_features = grafo_nx.nodes()[target_node]['x']


#assert grafo_nx.nodes[target_node]['x'] == flux_dict[target_node]
assert np.allclose(pyg_graph.x[producto_idx,:].numpy()[0:producto_features.__len__()], np.array(producto_features), 1e-7, 1e-10)
#assert np.allclose(flux_samples[target_node].tolist(), pyg_graph.x[producto_idx,:].numpy()[0:producto_features.__len__()], 1e-7, 1e-10)
assert not pyg_graph.is_directed()
assert not pyg_graph.has_isolated_nodes()


In [5]:
oversampled_metabolite_data         = pd.read_parquet("./results/data/oversampled_augmented_metabolite_data.parquet.gzip")#
labels                              = torch.tensor(oversampled_metabolite_data.label).reshape(len(oversampled_metabolite_data.label),1)
pyg_graph.y = labels

In [15]:
import torch
torch.save(pyg_graph, "./results/graphs_from_PYG_and_NX/PYG_graph_only_CONCEN.pt")



producto_features

In [16]:
from torch_geometric.data import Data

lista_de_grafos = []
for i in range(pyg_graph.x.shape[1]):
   
    
    new_pyg_data  = Data(x =  pyg_graph.x[:,i].reshape(12437, 1),  y = pyg_graph.y[i], 
                         edge_index = pyg_graph.edge_index)
    new_pyg_data.num_classes = int(2)
    lista_de_grafos.append(new_pyg_data)
    
    
from torch.utils.data import RandomSampler

def make_loader(graphs: list, batch_size: int  =1*32, num_samples = 500):#batch_size: int  =8*32):

    sampler_train_set = RandomSampler(
        graphs,
        #num_samples= num_samples, #params["training"]["sampler_num_samples"],  # Genera un muestreo del grafo
        replacement=True,  # con repeticion de muestras
    )
    return DataLoader(graphs, batch_size=batch_size, sampler = sampler_train_set,  drop_last=True)


loader   = make_loader(lista_de_grafos)

In [18]:
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GIN
from torch_geometric.nn import global_mean_pool
gc.collect()
torch.cuda.empty_cache()

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        
        self.GIN_layers =  GIN(in_channels= 1, hidden_channels= hidden_channels, num_layers= 4, 
                               out_channels= hidden_channels, dropout=0.1,  jk=None, 
                               act='LeakyReLU', act_first = True)   
        
        #self.conv1 = GCNConv(1, hidden_channels)
        #self.conv2 = GCNConv(hidden_channels, hidden_channels)
        #self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2, bias=True)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        #x = self.conv1(x, edge_index)
        #x = x.relu()
        x = self.GIN_layers(x, edge_index)
        #x = F.dropout(x, p=0.1)
        x = x.relu()
        #x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        
        x = self.lin(x)
        
        return F.log_softmax(x, dim=1) #x
    
model    = GCN(hidden_channels=8)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.NLLLoss()
def train():
    model.to('cuda')
    model.train()

    for data in loader:  # Iterate in batches over the training dataset.
         data.to('cuda')
         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()
     model.to('cuda')
     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         data.to('cuda')
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 10):
    train()
    train_acc = test(loader)
    test_acc = test(loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 001, Train Acc: 0.9550, Test Acc: 0.9634
Epoch: 002, Train Acc: 0.8896, Test Acc: 0.8888


In [15]:
nx.write_gpickle(grafo_nx, f"./results/graphs_from_PYG_and_NX/NX_only_concentrations.gpickle")

In [16]:
grafo_nx_v2 = copy.deepcopy(grafo_nx)

### add fluxes as features

In [18]:
flux_samples = pd.read_parquet( "./results/data/flux_samples.parquet.gzip")

In [29]:
feature_length = len(grafo_nx.nodes(data=True)['r0399']['x']) 


flux_dict = flux_samples.sample(feature_length, replace=True).to_dict(orient = 'list')

In [39]:
x_attribute     = nx.get_node_attributes(grafo_nx_v2, "x")

x_attribute.update(flux_dict)

nx.set_node_attributes(grafo_nx_v2, x_attribute, 'x')


assert grafo_nx_v2.nodes(data=True)['r0399']['x'] == flux_dict['r0399']
assert len(grafo_nx_v2.nodes(data=True)['r0399']['x']) == len(grafo_nx_v2.nodes(data=True)['phe_L_c']['x'])
assert grafo_nx_v2.nodes(data=True)['phe_L_c']['x']  == feature_data['phe_L_c'].tolist() 

In [40]:
nx.write_gpickle(grafo_nx_v2, f"./results/graphs_from_PYG_and_NX/NX_fluxes_and_concentrations.gpickle")